<a href="https://colab.research.google.com/github/markriis/IDS-pollution-of-cities/blob/main/Data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing data and librarys

In [36]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [37]:
gasses = pd.read_csv('https://raw.githubusercontent.com/owid/co2-data/master/owid-co2-data.csv',
                     usecols=['country', 'year', 'population', 'gdp', 'cement_co2',
       'cement_co2_per_capita', 'co2', 'co2_including_luc', 'co2_including_luc_growth_abs',
       'co2_including_luc_growth_prct', 'co2_including_luc_per_capita',
       'co2_including_luc_per_gdp', 'co2_including_luc_per_unit_energy',
       'co2_per_capita', 'co2_per_gdp', 'co2_per_unit_energy', 'coal_co2',
       'coal_co2_per_capita', 'cumulative_cement_co2', 'cumulative_co2',
       'cumulative_co2_including_luc', 'cumulative_coal_co2',
       'cumulative_flaring_co2', 'cumulative_gas_co2', 'cumulative_luc_co2',
       'cumulative_oil_co2', 'energy_per_capita',
       'energy_per_gdp', 'flaring_co2', 'flaring_co2_per_capita', 'gas_co2',
       'gas_co2_per_capita', 'ghg_excluding_lucf_per_capita', 'ghg_per_capita',
       'land_use_change_co2', 'land_use_change_co2_per_capita', 'methane',
       'methane_per_capita', 'nitrous_oxide', 'nitrous_oxide_per_capita',
       'oil_co2', 'oil_co2_per_capita', 'primary_energy_consumption',
       'total_ghg', 'total_ghg_excluding_lucf'])# Removed some columns that I doubt would be of use, meanings at https://github.com/owid/co2-data/blob/master/owid-co2-codebook.csv

air = pd.read_excel('https://cdn.who.int/media/docs/default-source/air-pollution-documents/air-quality-and-health/who_aap_2021_v9_11august2022.xlsx?sfvrsn=9035996c_3',
                    storage_options={'User-Agent': 'Mozilla/5.0'}, sheet_name='AAP_2022_city_v9',
                    usecols=['WHO Country Name', 'City or Locality', 'Measurement Year', 'PM2.5 (μg/m3)', 'PM10 (μg/m3)', 'NO2 (μg/m3)'])

development = pd.read_csv('https://hdr.undp.org/sites/default/files/2021-22_HDR/HDR21-22_Composite_indices_complete_time_series.csv',
                          storage_options={'User-Agent': 'Mozilla/5.0'},
                          usecols=['country', 'hdi_1990', 'hdi_1991', 'hdi_1992', 'hdi_1993',
       'hdi_1994', 'hdi_1995', 'hdi_1996', 'hdi_1997', 'hdi_1998',
       'hdi_1999', 'hdi_2000', 'hdi_2001', 'hdi_2002', 'hdi_2003',
       'hdi_2004', 'hdi_2005', 'hdi_2006', 'hdi_2007', 'hdi_2008',
       'hdi_2009', 'hdi_2010', 'hdi_2011', 'hdi_2012', 'hdi_2013',
       'hdi_2014', 'hdi_2015', 'hdi_2016', 'hdi_2017', 'hdi_2018',
       'hdi_2019', 'hdi_2020', 'hdi_2021'])#


populations = pd.read_csv('https://github.com/markriis/IDS-pollution-of-cities/raw/main/UNdata_Populations.zip',
                          usecols=['Country or Area', 'Year', 'City', 'Reliability', 'Value'],
                          nrows=27947)


Looking at datasets

In [38]:
air.head()

,WHO Country Name,City or Locality,Measurement Year,PM2.5 (μg/m3),PM10 (μg/m3),NO2 (μg/m3)
0,Afghanistan,Kabul,2019,119.77,NaN,NaN
1,Albania,Durres,2015,NaN,17.65,26.63
2,Albania,Durres,2016,14.32,24.56,24.78
3,Albania,Elbasan,2015,NaN,NaN,23.96
4,Albania,Elbasan,2016,NaN,NaN,26.26


In [39]:
development.head()

,country,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,hdi_1995,hdi_1996,hdi_1997,hdi_1998,...,hdi_2012,hdi_2013,hdi_2014,hdi_2015,hdi_2016,hdi_2017,hdi_2018,hdi_2019,hdi_2020,hdi_2021
0,Afghanistan,0.273,0.279,0.287,0.297,0.292,0.310,0.319,0.323,0.324,...,0.466,0.474,0.479,0.478,0.481,0.482,0.483,0.488,0.483,0.478
1,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.541,0.552,0.563,0.582,0.596,0.597,0.595,0.595,0.590,0.586
2,Albania,0.647,0.629,0.614,0.617,0.624,0.634,0.645,0.642,0.657,...,0.778,0.785,0.792,0.795,0.798,0.802,0.806,0.810,0.794,0.796
3,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.869,0.864,0.871,0.867,0.871,0.868,0.872,0.873,0.848,0.858
4,United Arab Emirates,0.728,0.739,0.742,0.748,0.755,0.762,0.767,0.773,0.779,...,0.846,0.852,0.859,0.865,0.870,0.897,0.909,0.920,0.912,0.911


In [40]:
gasses.head()

,country,year,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,...,land_use_change_co2_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,primary_energy_consumption,total_ghg,total_ghg_excluding_lucf
0,Afghanistan,1850,3752993.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.794,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,1851,3767956.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.797,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,1852,3783940.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,1853,3800954.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.799,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,1854,3818038.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
gasses.shape

(48058, 45)

In [42]:
populations.head()# I have no idea why some names are fully in caps

,Country or Area,Year,City,Reliability,Value
0,Åland Islands,2022,MARIEHAMN,"Final figure, complete",11750.0
1,Åland Islands,2021,MARIEHAMN,"Final figure, complete",11724.0
2,Åland Islands,2020,MARIEHAMN,"Final figure, complete",11692.0
3,Åland Islands,2019,MARIEHAMN,"Final figure, complete",11711.0
4,Åland Islands,2018,MARIEHAMN,"Final figure, complete",11709.0


In [43]:
populations['Value'] = populations['Value'].astype(int)# Number of people should be whole

populations = populations[populations['Reliability'] == 'Final figure, complete'].drop(columns = ['Reliability'])# Most of the data is final figure, I think we can skip the rest
populations.reset_index(inplace=True, drop=True)
populations.head()

,Country or Area,Year,City,Value
0,Åland Islands,2022,MARIEHAMN,11750
1,Åland Islands,2021,MARIEHAMN,11724
2,Åland Islands,2020,MARIEHAMN,11692
3,Åland Islands,2019,MARIEHAMN,11711
4,Åland Islands,2018,MARIEHAMN,11709


Finding what countries are in all dataframes

In [44]:
countries_air = air['WHO Country Name'].dropna().values
countries_dev = development['country'].dropna().values
countries_gas = gasses['country'].dropna().values
countries_pop = populations['Country or Area'].dropna().values

display(pd.unique(countries_air).shape)# It apears air has the least countries in it, but still 118 is prety good
display(pd.unique(countries_dev).shape)
display(pd.unique(countries_gas).shape)
display(pd.unique(countries_pop).shape)

(118,)

(206,)

(264,)

(216,)

In [45]:
common_countries = np.intersect1d(np.intersect1d(countries_air, countries_dev), np.intersect1d(countries_gas, countries_pop))
display(common_countries)
display(common_countries.shape)

array(['Albania', 'Algeria', 'Andorra', 'Argentina', 'Australia',
       'Austria', 'Bahamas', 'Bahrain', 'Bangladesh', 'Belgium', 'Bhutan',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Cameroon',
       'Canada', 'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia',
       'Cuba', 'Czechia', 'Denmark', 'Ecuador', 'Egypt', 'El Salvador',
       'Estonia', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Georgia',
       'Germany', 'Ghana', 'Greece', 'Guatemala', 'Honduras', 'Hungary',
       'Iceland', 'India', 'Indonesia', 'Iraq', 'Ireland', 'Israel',
       'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya',
       'Kuwait', 'Kyrgyzstan', 'Latvia', 'Lebanon', 'Liechtenstein',
       'Lithuania', 'Luxembourg', 'Madagascar', 'Malaysia', 'Maldives',
       'Malta', 'Mauritius', 'Mexico', 'Monaco', 'Mongolia', 'Montenegro',
       'Myanmar', 'Nepal', 'Netherlands', 'New Zealand', 'Nigeria',
       'North Macedonia', 'Norway', 'Pakistan', 'Panama', 'Paraguay',
       'Peru'

(103,)

In [46]:
air.rename(columns={'WHO Country Name' : 'country', 'City or Locality' : 'city'}, inplace=True)
populations.rename(columns={'Country or Area' : 'country', 'City' : 'city'}, inplace=True)

datasets = {
    'gasses' : gasses,
    'air' : air,
    'development' : development,
    'populations' : populations
}

In [47]:
for name, dataset in datasets.items():
  datasets[name] = dataset[dataset['country'].isin(common_countries)]
  datasets[name].reset_index(inplace=True, drop=True)

In [48]:
datasets['gasses'] = datasets['gasses'][datasets['gasses']['year'] >= 1990]

In [49]:
for name, dataset in datasets.items():
  display(dataset.shape)

(3399, 45)

(28512, 6)

(103, 33)

(19303, 4)

Finding the cities that are in both air and population dataframes

In [54]:
air_country_cities = datasets['air']['country'] + '_' + datasets['air']['city']# Compareing city-country pairs encase there are several citys with same name
pop_country_cities = datasets['populations']['country'] + '_' + datasets['populations']['city']


In [53]:
def sName(df):# A function to get rid of diferences in the names that are most likely to ocurr, because of human error
  return df.str.lower().str.replace(' |-', '', regex=True)

In [55]:
common_idx = np.intersect1d(sName(air_country_cities), sName(pop_country_cities), return_indices=True)[1]
common_idx

array([   19,    25,    54, ..., 28361, 28363, 28365])

In [56]:
common_cities = datasets['air']['city'].iloc[common_idx]

datasets['air'] = datasets['air'][datasets['air']['city'].isin(common_cities)] #
datasets['air'].reset_index(inplace=True, drop=True)

datasets['populations'] = datasets['populations'][sName(datasets['populations']['city']).isin(sName(common_cities))]
datasets['populations'].reset_index(inplace=True, drop=True)


In [57]:
for name, dataset in datasets.items():
  display(dataset.shape)

(3399, 45)

(6145, 6)

(103, 33)

(7958, 4)

Getting the city names in two datasets to match

In [63]:
datasets['populations'].loc[~datasets['populations']['city'].isin(common_cities), 'city'] = datasets['populations'].loc[~datasets['populations']['city'].isin(common_cities), 'city'].str.title()# Get rid of comleet uppercase

In [64]:
datasets['populations'][~datasets['populations']['city'].isin(common_cities)]['city'].unique()

array(['Guangan', 'Huaian', 'Linan', 'Weinan', 'Hubli-Dharwad',
       'Pimpri Chinchwad', 'Rae Bareli', 'Thiruvananthapuram',
       "Reggio Nell'Emilia", 'Vacoas - Phoenix', 'Bielsko-Biala',
       'Satu-Mare', 'Drobeta Turnu-Severin', "L'Hospitalet De Llobregat",
       'Port-Of-Spain'], dtype=object)

In [66]:
for city in datasets['populations'][~datasets['populations']['city'].isin(common_cities)]['city'].unique(): #Iterate through the city names that still dont match

  location = sName(common_cities)[sName(common_cities) == (city.lower().replace('-', '').replace(' ', ''))].index[0]
  datasets['populations'].loc[datasets['populations']['city'] == city, 'city'] = common_cities[location]
  print(f'Replaced {city} with {str(common_cities[location])} from common_cities')
  #datasets['populations'].loc[i, 'ASCII city'] = common_cities[location]


Replaced Guangan with Guang An from common_cities
Replaced Huaian with Huai An from common_cities
Replaced Linan with Lin An from common_cities
Replaced Weinan with Wei Nan from common_cities
Replaced Hubli-Dharwad with Hubli- Dharwad from common_cities
Replaced Pimpri Chinchwad with Pimpri- Chinchwad from common_cities
Replaced Rae Bareli with Raebareli from common_cities
Replaced Thiruvananthapuram with Thiruvanantha Puram from common_cities
Replaced Reggio Nell'Emilia with Reggio Nell'emilia from common_cities
Replaced Vacoas - Phoenix with Vacoas-Phoenix from common_cities
Replaced Bielsko-Biala with Bielsko Biala from common_cities
Replaced Satu-Mare with Satu Mare from common_cities
Replaced Drobeta Turnu-Severin with Drobeta Turnu Severin from common_cities
Replaced L'Hospitalet De Llobregat with L'hospitalet De Llobregat from common_cities
Replaced Port-Of-Spain with Port Of Spain from common_cities


In [67]:
datasets['populations'][~datasets['populations']['city'].isin(common_cities)].shape

(0, 4)

A look at the processed data

In [68]:
for name, dataset in datasets.items():
  display(dataset)

,country,year,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_including_luc,co2_including_luc_growth_abs,co2_including_luc_growth_prct,...,land_use_change_co2_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,oil_co2,oil_co2_per_capita,primary_energy_consumption,total_ghg,total_ghg_excluding_lucf
140,Albania,1990,3295073.0,1.294785e+10,0.384,0.116,5.521,7.287,-3.556,-32.796,...,0.536,3.55,1.077,1.57,0.476,2.173,0.659,33.658,11.37,11.18
141,Albania,1991,3302087.0,9.375844e+09,0.307,0.093,4.290,5.605,-1.683,-23.093,...,0.398,3.50,1.060,1.20,0.363,2.228,0.675,28.230,9.01,8.82
142,Albania,1992,3303738.0,8.745681e+09,0.103,0.031,2.517,3.777,-1.827,-32.604,...,0.381,3.41,1.032,1.11,0.336,1.416,0.429,24.674,6.97,6.78
143,Albania,1993,3300715.0,9.636359e+09,0.103,0.031,2.338,3.184,-0.593,-15.704,...,0.256,3.49,1.057,1.16,0.351,1.478,0.448,21.638,6.94,6.75
144,Albania,1994,3294001.0,1.059613e+10,0.052,0.016,1.924,2.615,-0.569,-17.861,...,0.210,4.08,1.239,1.38,0.419,1.605,0.487,20.567,7.75,7.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18601,Uzbekistan,2018,32449578.0,3.368748e+11,3.776,0.116,102.288,100.672,-6.584,-6.139,...,-0.050,54.25,1.672,16.62,0.512,11.692,0.360,533.657,181.47,184.84
18602,Uzbekistan,2019,32976946.0,NaN,4.511,0.137,108.429,107.044,6.372,6.329,...,-0.042,54.44,1.651,16.78,0.509,11.314,0.343,534.630,186.84,190.21
18603,Uzbekistan,2020,33526662.0,NaN,5.147,0.154,109.632,108.379,1.335,1.247,...,-0.037,53.60,1.599,16.99,0.507,12.696,0.379,531.753,184.15,187.52
18604,Uzbekistan,2021,34081452.0,NaN,6.157,0.181,116.388,115.257,6.878,6.346,...,-0.033,NaN,NaN,NaN,NaN,14.700,0.431,564.146,NaN,NaN


,country,city,Measurement Year,PM2.5 (μg/m3),PM10 (μg/m3),NO2 (μg/m3)
0,Argentina,Buenos Aires,2014,10.11,NaN,NaN
1,Argentina,Buenos Aires,2015,10.26,27.87,NaN
2,Argentina,Buenos Aires,2016,NaN,26.88,15.35
3,Argentina,Buenos Aires,2017,NaN,25.40,19.57
4,Argentina,Buenos Aires,2018,NaN,24.40,16.60
...,...,...,...,...,...,...
6140,Ukraine,Kyiv,2019,23.36,31.19,NaN
6141,Uruguay,Montevideo,2014,8.00,25.83,NaN
6142,Uruguay,Montevideo,2018,12.00,19.00,NaN
6143,Uzbekistan,Tashkent,2018,43.09,NaN,NaN


,country,hdi_1990,hdi_1991,hdi_1992,hdi_1993,hdi_1994,hdi_1995,hdi_1996,hdi_1997,hdi_1998,...,hdi_2012,hdi_2013,hdi_2014,hdi_2015,hdi_2016,hdi_2017,hdi_2018,hdi_2019,hdi_2020,hdi_2021
0,Albania,0.647,0.629,0.614,0.617,0.624,0.634,0.645,0.642,0.657,...,0.778,0.785,0.792,0.795,0.798,0.802,0.806,0.810,0.794,0.796
1,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.869,0.864,0.871,0.867,0.871,0.868,0.872,0.873,0.848,0.858
2,Argentina,0.723,0.730,0.735,0.739,0.744,0.745,0.751,0.756,0.762,...,0.843,0.845,0.846,0.848,0.847,0.851,0.850,0.852,0.840,0.842
3,Australia,0.865,0.867,0.869,0.873,0.873,0.882,0.884,0.887,0.891,...,0.930,0.929,0.931,0.933,0.935,0.937,0.941,0.941,0.947,0.951
4,Austria,0.825,0.829,0.835,0.840,0.845,0.849,0.853,0.857,0.863,...,0.906,0.905,0.909,0.910,0.915,0.916,0.917,0.919,0.913,0.916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,Uganda,0.329,0.331,0.325,0.332,0.336,0.342,0.354,0.364,0.373,...,0.504,0.509,0.512,0.517,0.519,0.520,0.522,0.525,0.524,0.525
99,Ukraine,0.729,0.723,0.715,0.703,0.689,0.684,0.685,0.689,0.696,...,0.773,0.773,0.773,0.774,0.779,0.782,0.783,0.786,0.775,0.773
100,Uruguay,0.701,0.705,0.708,0.711,0.716,0.717,0.723,0.735,0.744,...,0.798,0.805,0.808,0.811,0.815,0.819,0.819,0.821,0.821,0.809
101,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.687,0.693,0.698,0.701,0.709,0.715,0.720,0.726,0.721,0.727


,country,Year,city,Value
0,Argentina,2022,Buenos Aires,15716718
1,Argentina,2021,Buenos Aires,15567820
2,Argentina,2020,Buenos Aires,15416728
3,Argentina,2019,Buenos Aires,15263440
4,Argentina,2016,Buenos Aires,13879707
...,...,...,...,...
7953,Uzbekistan,2018,Tashkent,2464933
7954,Uzbekistan,2017,Tashkent,2424146
7955,Uzbekistan,2016,Tashkent,2393176
7956,Uzbekistan,2015,Tashkent,2371269


In [69]:
g = datasets['populations'].groupby(['country', 'city'])

g.min()

Year    Value
country    city                        
Argentina  Buenos Aires   1991  2965403
Australia  Adelaide       2001    24193
           Brisbane       2001  1209322
           Canberra       2001   308764
           Central Coast  2001   254579
...                        ...      ...
Tunisia    Tunis          2014  1056247
Uganda     Kampala        2002  1208544
Ukraine    Kyiv           2001  2566953
Uruguay    Montevideo     2003  1305082
Uzbekistan Tashkent       2001  2137218

[1106 rows x 2 columns]

In [70]:
g.count()

Year  Value
country    city                      
Argentina  Buenos Aires     11     11
Australia  Adelaide         17     17
           Brisbane         17     17
           Canberra         12     12
           Central Coast    10     10
...                        ...    ...
Tunisia    Tunis             1      1
Uganda     Kampala           2      2
Ukraine    Kyiv             10     10
Uruguay    Montevideo       15     15
Uzbekistan Tashkent          9      9

[1106 rows x 2 columns]